In [1]:
from collections import Counter
import numpy as np
import tensorflow.contrib.keras as kr
import tensorflow as tf
import time
from datetime import timedelta
import os
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"
savePath = "s3://corpus-2/model/ShinContextual"

In [3]:
def loadGloVe(filename, emb_size=50):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])

        for i in range(num):
            np.random.shuffle(train_data[i])
            train_data[i] = np.asarray(train_data[i])

        np.random.shuffle(train_data)   
        return train_data
    
    
    train_data = []
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, split_type):
    
    print(split_type)
    
    train_acc = []
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)

        print(len(tx),len(tx[0]),len(tx[1]),len(tx[0][0]))
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(tx[test_i],ty[test_i])
            train_acc.append(model_train(train_x, train_y,split_type,fold_id))
            test_acc.append(model_evaluate(test_x, test_y,split_type,fold_id,categories))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_acc.append(model_train(tx[train_i], ty[train_i],split_type,fold_id))
            test_acc.append(model_evaluate(tx[test_i], ty[test_i],split_type,fold_id,categories))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
class ShinContextualModel:
    '''
    Implementation proposal of: http://milab.snu.ac.kr/pub/BigComp2018.pdf
    '''
    def __init__(self, embedding, drop_rate=0.75, conv_size=3, num_layers=2):
        '''Constructor.
        # Parameters:
        embedding: numpy array representing the embedding.
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        num_layers: Number of recurrent convolutions.
        '''
        self._embedding = embedding
        self._conv_size = conv_size
        self._num_layers = num_layers
        self.drop_rate = drop_rate

    def __call__(self, input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, input)
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size, self._num_layers, self._embedding_tf)
        self._pooling_tf = self._create_maxpooling_layer(self._convolution_tf)

        return self._pooling_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('pool:', str(self._pooling_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self, conv_size, num_layers, embedding):
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, filter_width]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[filter_width]))

        z = embedding
        for _ in range(num_layers):
            conv = tf.nn.conv1d(
                value=z,
                filters=W,
                stride=1,
                padding='SAME')
            bias = tf.nn.bias_add(conv, b)
            c = tf.nn.relu(bias)

            d = tf.nn.dropout(c, self.drop_rate)
            # Add BatchNormalization or LocalResponseNormalization
            e = tf.expand_dims(d, 1)

            z = tf.nn.local_response_normalization(
                e,
                depth_radius=5,
                bias=1,
                alpha=0.001,
                beta=0.75
            )
            z = tf.squeeze(z, 1)
        # endfor
        return z

    def _create_maxpooling_layer(self, convolution):
        conv_size = convolution.shape[1].value
        embedding_size = convolution.shape[2].value

        convolution = tf.expand_dims(convolution, -1)
        pooled = tf.nn.max_pool(
            value=convolution,
            ksize=[1, conv_size, 1, 1],
            strides=[1, 1, 1, 1],
            padding='VALID')

        flat = tf.reshape(pooled, [-1, embedding_size])
        return flat

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

In [6]:
seq_length = 41

In [7]:
vocab, embd = loadGloVe(vocabPath, 100)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
# embedding = np.asarray(embd)
embedding = embd

Loading GloVe!
Completed!


In [8]:
word_to_id = dict(zip(vocab, range(vocab_size)))
len(embedding),embedding_dim,vocab_size

(400000, 100, 400000)

In [9]:
def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [10]:
learning_rate = 1e-3
dropout_keep_prob = 0.75

# 输入内容及对应的标签
input_x = tf.placeholder(tf.int32, [None, seq_length], name='input_x')
input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

model = ShinContextualModel(embedding, drop_rate = keep_prob)
fc = model(input_x)
model.summary()

# 分类器
logits = tf.layers.dense(fc, num_classes, name='fc2')
y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）

# 损失函数，交叉熵
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=input_y)
loss = tf.reduce_mean(cross_entropy)
# 优化器
optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# 准确率
correct_pred = tf.equal(tf.argmax(input_y, 1), y_pred_cls)
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


saver = tf.train.Saver()

embedding: (?, 41, 100)
conv: (?, 41, 100)
pool: (?, 100)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, x_pad, y_pad, loss1, acc1, batch_size):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {input_x: x_batch1, input_y: y_batch1, keep_prob: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len


def model_train(x_train, y_train, split_type, fold_id):
    
    save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 创建session
    session = tf.Session()
    session.run(tf.global_variables_initializer())

    print('Training and evaluating...')
    start_time = time.time()
    total_batch = 0  # 总批次
    best_acc_train = 0.0  # 最佳验证集准确率
    last_improved = 0  # 记录上一次提升批次
    num_epochs = 50
    batch_size = 32
    require_improvement = 500  # 如果超过1000轮未提升，提前结束训练
    print_per_batch = 30  # 每多少轮输出一次结果
    flag = False

    for epoch in range(num_epochs):  # 20
        print('Epoch:', epoch + 1)
        batch_train = batch_iter(x_train, y_train, batch_size)
        for x_batch, y_batch in batch_train:
            feed_dict = {input_x: x_batch, input_y: y_batch, keep_prob: dropout_keep_prob}
            session.run(optim, feed_dict=feed_dict)  # 运行优化
            total_batch += 1

            if total_batch % print_per_batch == 0:
                # 每多少轮次输出在训练集和验证集上的性能
                feed_dict[keep_prob] = 1.0
                loss_train, acc_train = session.run([loss, acc], feed_dict=feed_dict)
                # loss_val, acc_val = evaluate(session, x_dev, y_dev, loss, acc)
                if acc_train > best_acc_train:
                    # 保存最好结果
                    best_acc_train = acc_train
                    last_improved = total_batch
                    saver.save(sess=session, save_path=save_path)
                    improved_str = '*'
                else:
                    improved_str = ''

                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%}, Time: {3} {4}'
                print(msg.format(total_batch, loss_train, acc_train, time_dif, improved_str))

            if total_batch - last_improved > require_improvement:
                # 验证集正确率长期不提升，提前结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break  # 跳出循环
        if flag:  # 同上
            break

    session.close()
    return best_acc_train


def model_evaluate(x_test, y_test, split_type, fold_id, categories, batch_size=32):
        
    save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 读取保存的模型
    session = tf.Session()
    saver.restore(sess=session, save_path=save_path)
    start_time = time.time()
    print('Testing...')
    loss_test, acc_test = evaluate(session, x_test, y_test, loss, acc, batch_size)
    msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
    print(msg.format(loss_test, acc_test))

    test_data_len = len(x_test)
    test_num_batch = int((test_data_len - 1) / batch_size) + 1

    y_test_cls = np.argmax(y_test, 1)  # 获得类别
    y_test_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

    for i in range(test_num_batch):  # 逐批次处理
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, test_data_len)
        feed_dict = {
            input_x: x_test[start_id:end_id],
            keep_prob: 1.0
        }
        y_test_pred_cls[start_id:end_id] = session.run(y_pred_cls, feed_dict=feed_dict)

    # 评估
    print("Precision, Recall and F1-Score...")
    print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
    '''
    sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
        y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
        y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
        labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
        target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
        原文链接：https://blog.csdn.net/akadiao/article/details/78788864
    '''

    # 混淆矩阵
    print("Confusion Matrix...")
    cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
    '''
    混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
    这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
    https://blog.csdn.net/u011734144/article/details/80277225
    '''
    print(cm)

    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)
    session.close()

    return acc_test

In [12]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, split_type))

random
14035 41 41
Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    2.3, Train Acc:  15.62%, Time: 0:00:15 *
Iter:     60, Train Loss:    2.1, Train Acc:  28.12%, Time: 0:00:25 *
Iter:     90, Train Loss:    2.1, Train Acc:  21.88%, Time: 0:00:25 
Iter:    120, Train Loss:    2.0, Train Acc:  40.62%, Time: 0:00:35 *
Iter:    150, Train Loss:    1.9, Train Acc:  43.75%, Time: 0:00:46 *
Iter:    180, Train Loss:    1.9, Train Acc:  40.62%, Time: 0:00:46 
Iter:    210, Train Loss:    1.7, Train Acc:  43.75%, Time: 0:00:46 
Iter:    240, Train Loss:    1.4, Train Acc:  59.38%, Time: 0:00:56 *
Iter:    270, Train Loss:    1.3, Train Acc:  75.00%, Time: 0:01:06 *
Iter:    300, Train Loss:    1.3, Train Acc:  59.38%, Time: 0:01:06 
Iter:    330, Train Loss:    1.0, Train Acc:  78.12%, Time: 0:01:15 *
Iter:    360, Train Loss:    1.1, Train Acc:  59.38%, Time: 0:01:15 
Iter:    390, Train Loss:   0.88, Train Acc:  87.50%, Time: 0:01:25 *
Epoch: 2
Iter:    420, Train Lo

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/ShinContextual/random/1/1


Testing...
Test Loss:   0.43, Test Acc:  87.32%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.84      0.95      0.89       142
                   Filter       0.76      0.88      0.82       143
    Compute Derived Value       0.81      0.80      0.81       142
            Find Extremum       0.95      0.85      0.90       157
                     Sort       0.93      0.90      0.92       110
          Determine Range       0.88      0.83      0.85       135
Characterize Distribution       0.90      0.87      0.88       139
           Find Anomalies       0.89      0.84      0.87       126
                  Cluster       0.86      0.91      0.88       131
                Correlate       0.94      0.90      0.92       179

                micro avg       0.87      0.87      0.87      1404
                macro avg       0.88      0.87      0.87      1404
             weighted avg       0.88      0.8

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/ShinContextual/random/2/2


Testing...
Test Loss:   0.33, Test Acc:  90.17%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.94      0.96      0.95       127
                   Filter       0.82      0.93      0.87       148
    Compute Derived Value       0.96      0.88      0.92       154
            Find Extremum       0.89      0.98      0.93       176
                     Sort       0.90      0.91      0.90       107
          Determine Range       0.88      0.84      0.86       158
Characterize Distribution       0.95      0.87      0.91       127
           Find Anomalies       0.85      0.86      0.86       147
                  Cluster       0.96      0.90      0.93       117
                Correlate       0.92      0.88      0.90       143

                micro avg       0.90      0.90      0.90      1404
                macro avg       0.91      0.90      0.90      1404
             weighted avg       0.90      0.9

...

Testing...
Test Loss:    1.1, Test Acc:  67.27%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.41      0.76      0.54       104
                   Filter       0.63      0.63      0.63       110
    Compute Derived Value       0.49      0.50      0.49        86
            Find Extremum       0.74      0.63      0.68       133
                     Sort       0.94      0.74      0.83        89
          Determine Range       0.83      0.56      0.67        86
Characterize Distribution       0.76      0.67      0.71       116
           Find Anomalies       0.78      0.60      0.68        89
                  Cluster       0.77      0.82      0.79       120
                Correlate       0.70      0.73      0.72       170

                micro avg       0.67      0.67      0.67      1103
                macro avg       0.71      0.66      0.67      1103
             weighted avg       0.71      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/ShinContextual/table/9/9


Testing...
Test Loss:    1.2, Test Acc:  62.57%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.31      0.31      0.31        51
                   Filter       0.70      0.17      0.27        95
    Compute Derived Value       0.57      0.65      0.61        83
            Find Extremum       0.41      0.97      0.58        68
                     Sort       0.66      0.64      0.65        64
          Determine Range       0.78      0.26      0.39        80
Characterize Distribution       0.79      0.88      0.83        68
           Find Anomalies       0.79      0.81      0.80        80
                  Cluster       0.68      0.88      0.77        76
                Correlate       0.91      0.78      0.84        67

                micro avg       0.63      0.63      0.63       732
                macro avg       0.66      0.64      0.61       732
             weighted avg       0.67      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/ShinContextual/table/10/10


Testing...
Test Loss:    1.1, Test Acc:  62.75%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.50      0.56      0.53        79
                   Filter       0.45      0.21      0.29        91
    Compute Derived Value       0.67      0.40      0.50        94
            Find Extremum       0.74      0.99      0.85        84
                     Sort       0.41      0.97      0.58        29
          Determine Range       0.53      0.51      0.52        73
Characterize Distribution       0.49      0.85      0.62        26
           Find Anomalies       0.68      0.90      0.78        77
                  Cluster       0.83      0.70      0.76        54
                Correlate       0.87      0.66      0.75        91

                micro avg       0.63      0.63      0.63       698
                macro avg       0.62      0.67      0.62       698
             weighted avg       0.64      0.6